**线性回归**

- 损失函数
$$J(\theta) = \frac{1}{2}(\bar{y}_i - y_i)^2 = \frac{1}{2} \sum_{i=1}^n (h_{\theta}(x_i) - y_i)^2$$

- 引入矩阵运算
$$J(\theta) = \frac{1}{2} (X \theta - Y)^T (X \theta - Y) = \frac{1}{2}(\theta ^ T X^T - Y^T)(X \theta - Y)=\frac{1}{2} (\theta ^ T X^T X \theta - 2\theta ^T X^T Y + Y^T Y)$$

- 矩阵求导运算
$$\frac{dAB}{dB} = A^T$$

$$\frac{dA^TB}{A} = B$$

$$\frac{dX^T AX}{dX} = 2AX$$

- 对$\theta$进行求导

$$\frac{\partial J(\theta)}{\partial \theta} = \frac{1}{2} (2X^TX \theta - 2X^T \theta)=X^TX \theta - X^TY$$

- 令 $\frac{\partial J(\theta)}{\partial \theta} = 0$

- 可以得到

$$\theta = (X^TX)^{-1}X^T Y$$

In [2]:
import numpy as np

In [3]:
class LinearRegression:
    def __init__(self, fit_intercept=True):
        """
        最小二乘回归模型
        参数：
        @ fit_intercept : bool
            截距是否存在即线性函数是否含有b
        """
        self.beta = None
        self.fit_intercept = fit_intercept
        
    def fit(self, X, y):
        """
        通过极大似然拟合回归系数
        参数：
        @X : 形状大小为'(N, M)'，即N个样本，每个样本M个维度
        @y ：形状大小为'(N, K)'，即N个样本，每个target有K维
        """
        
        if self.fit_intercept:
            X = np.c_[np.ones(X.shape[0]), X]  # 添加1列到X中，作为偏置/截距的系数
            
        pseudo_inverse = np.dot(np.linalg.inv(np.dot(X.T, X)), X.T)
        self.beta = np.dot(pseudo_inverse, y)
        
    def predict(self, X):
        """
        用生成的模型预测
        参数：
        @X : 数据形状为'(Z, M)'，即数据集是由Z个样本组成，样本维度为M
        返回值：
        @y_pred = 数据形状为'(Z, K)'
        """
        
        if self.fit_intercept:
            X = np.c_[np.ones(X.shape[0]), X]
        return np.dot(X, self.beta)

**Ridge回归**

在线性回归的基础上，加入了'l2'惩罚项

In [5]:
class RidgeRegression:
    def __init__(self, alpha=1, fit_intercept=True):
        self.beta = None
        self.alpha = alpha
        self.fit_intercept = fit_intercept
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = np.c_[np.ones(X.shape[0]), X]
        
        A = self.alpha * np.eye(X.shape[1])
        pseudo_inverse = np.dot(np.linalg.inv(X.T @ X + A), X.T)
        self.beta = pseudo_inverse @ y
    
    def predict(self, X):
        if self.fit_intercept:
            X = np.c_[np.ones(X.shape[0]), X]
        return np.dot(X, self.beta)

**Logisitic Regression** 的基本函数为sigmoid函数，即$\frac{1}{1+e^{-z}}$。

其中线性函数定义为: $z_{\theta}(x)=\theta_0x_0 + \theta_1 x_1 + \cdots + \theta_n x_n = \theta^T x$，逻辑回归函数是在线性函数的基础上添加了一个非线性映射，即
$$h_{\theta}(x)=\frac{1}{1+e^{-z_{\theta}(x)}} = \frac{1}{1+e^{-\theta^Tx}}$$

对于二分类来讲，用$\theta$来表示为正样本的概率即，
$$h_{\theta}(x)=\frac{1}{1+e^{-z_{\theta}(x)}}=\frac{1}{1+e^{-\theta^Tx}}$$
同理，表示负样本为：
$$h_{\theta}(x)=\frac{e^{-z_{\theta}(x)}}{1+e^{-z_{\theta}(x)}}=\frac{e^{-\theta^Tx}}{1+e^{-\theta^Tx}}$$

通常，利用阈值进行正负样本的预测或者多分类问题即 $\bar{y}=1$ if $\bar{h} \geq 0.5$, and $y = 0$ if $\bar{h} < 0.5$. The **decision boundary** is given by $\theta^T \cdot x=0$. 

逻辑回归的损失函数为logloss或者交叉熵损失函数:
$$J(\theta)=-\frac{1}{m} \sum_{i=1}^m[y^{(i)}log(h_{\theta}^{(i)})+(1-y^{(i)})log(1-h_{\theta}^{(i)})]$$
求导之后的损失:
$$\frac{\partial J(\theta)}{\partial \theta_j}=\frac{1}{m}\sum_{i=1}^m(h^{(i)}-y^{(i)})x_j^{(i)}$$

手动推导:
$$
\begin{equation}
\begin{aligned}
h^{\prime} &= (\frac{1}{1+e^{-\theta^Tx}})^{\prime} \\
& = - \frac{1}{(1+e^{-\theta^Tx})^2} \cdot (1+e^{-\theta^Tx})^{\prime} \\
& = - \frac{1}{(1+e^{-\theta^Tx})^2} \cdot e^{-\theta^Tx} \cdot (-\theta^Tx)^{\prime} \\
& = - \frac{xe^{-\theta^Tx}}{(1+e^{\theta^Tx})^2} \\
& = - \frac{1}{1+e^{-\theta^Tx}} \cdot \frac{e^{-\theta^Tx}}{1+e^{-\theta^Tx}} \cdot x \\
& = h(1-h)x
\end{aligned}
\end{equation}
$$

$$
\begin{equation}
\begin{aligned}
J(\theta)^{\prime} &= \sum_{i=1}^m(y_i log^{\prime}(h) + (1-y_i)log^{\prime}(1-h))) \\
& = \sum ((y_i\frac{1}{h}h^{\prime})+(1-y_i)\frac{1}{1-h}(1-h)^{\prime}) \\
& = \sum (y_i (1-h)x_i - (1-y_i)hx_i )\\
& = \sum (y_i-h)x_i
\end{aligned}
\end{equation}
$$
where $i$ means $i$-th sample, and $j$ means $j$-th feature.

In [6]:
def sigmoid(x):
    return 1. / (1. + np.exp(x))

In [8]:
class LogisticRegression:
    def __init__(self, penalty='l2', gamma=0, fit_intercept=True):
        err_msg = "penalty must be 'l1' or 'l2', but got: {}".format(penalty)
        assert penalty in ['l1', 'l2'], err_msg
        
        self.beta = None
        self.gamma = gamma
        self.penalty = penalty
        self.fit_intercept = fit_intercept
        
    def fit(self, X, y, lr=0.01, tol=1e-7, max_iter=1e7):
        if self.fit_intercept:
            X = np.c_[np.ones(X.shape[0]), X]
        
        l_prev = np.inf
        
        self.beta = np.random.rand(X.shape[1])
        for _ in range(int(max_iter)):
            y_pred = sigmoid(np.dot(X, self.beta))
            loss = self._NLL(X, y, y_pred)
            if l_prev - loss < tol:
                return
            l_prev = loss
            self.beta -= self.lr * self._NLL_grad(X, y, y_pred)
            
    def _NLL(self, X, y, y_pred):
        N, M = X.shape
        order = 2 if self.penalty == 'l2' else 1
        nll = -np.log(y_pred[y==1]).sum() - np.log(1-y_pred[y==0]).sum()
        penalty = 0.5 * self.gamma * np.linalg.norm(self.beta, ord=order) ** 2
        return (nll + penalty) / N
    
    def _NLL_grad(self, X, y, y_pred):
        N, M = X.shape
        p = self.penalty
        beta = self.beta
        gamma = self.gamma
        
        l1norm = lambda x : np.linalg.norm(x, 1)
        d_penalty = gamma * beta if p == 'l2' else gamma * l1norm(beta) * np.sign(beta)  # 绝对值求导 https://www.zhihu.com/question/276554773
        return -(np.dot(y - y_pred, X) + d_penalty) / N 
        
    def predict(self, X):
        if self.fit_intercept:
            X = np.c_[np.ones(X.shape[0]), X]
        return sigmoid(np.dot(X, self.beta))